In [ ]:
mpans = input("Please input the MPANs to run as csv format :: ")

In [1]:
print("Query")

d10_query = f"""SELECT
  wh.mpan AS MPAN
  , wh.serial_number AS MSN
  , wh.supplier_id AS SUPPLIER
  , st.latest_dc_name AS DC
  , TO_DATE(missingcos.relevant_supply_period_end_london_tz) AS EFD
  , dr.market_supply_point_id
  , dr.device_id
  , dr.register_id
  , MAX_BY(dr.read_value, dr.read_on) AS latest_reading
  , MAX(dr.read_on) AS latest_read_at
  , wh.standard_settlement_configuration_id AS SSC
FROM consumer.fnl_industry_missingcosreads AS missingcos
  JOIN octoenergy_data_prod_prod.amphio.inter_electricity_accumulation AS dr
    ON missingcos.mpan = dr.market_supply_point_id
  JOIN consumer.wh_electricity_validated_register_configuration_agreement_rate_interval AS wh
    ON wh.mpan = missingcos.mpan
    AND wh.identifier = dr.register_id
  JOIN consumer.fnl_industry_settlement AS st
    ON missingcos.mpan = st.mpan
WHERE dr.market_supply_point_id IN ({mpans})
GROUP BY dr.market_supply_point_id, dr.device_id, dr.register_id, wh.MPAN, MSN, SUPPLIER, DC, EFD, SSC
ORDER BY dr.market_supply_point_id DESC, dr.device_id DESC, dr.register_id ASC;"""

with tio.db(kraken) as query_2:
    d10_query_results = query_2.get_df(d10_query)
display(d10_query_results)
d10_query_results.to_csv('/home/jovyan/shared/Settlement/SSC Changer/D10_variables.csv')

Query


NameError: name 'mpans' is not defined

In [ ]:
# ***** functions for writing d52 / d205 / d367 / cmex *****
print("^ Functions")

def generate_single_d10_content (MPAN, MSN, formatted_EFD, d10_body_output, d10_body_counter, read_1):
    d10_content = f"""026|{MPAN}|U|\n028|{MSN}|R|\n030|1|{formatted_EFD}|{read_1}||||N|"""
    
    with open(d10_body_output, 'a')as d10_body:
        d10_body.write(d10_content)
        d10_body.write('\n')
    
    d10_body_counter [0] += 1
    
    
def generate_dual_d10_content (MPAN, MSN, formatted_EFD, d10_body_output, d10_body_counter, read_1, read_2):
    d10_content = f"""026|{MPAN}|U|\n028|{MSN}|R|\n030|1|{formatted_EFD}|{read_1}||||N|\n030|2|{formatted_EFD}|{read_2}||||N|"""
    
    with open(d10_body_output, 'a')as d10_body:
        d10_body.write(d10_content)
        d10_body.write('\n')
    
    d10_body_counter [0] += 1
    

def generate_d10_headfoot(formatted_dt, d10_counter, d10_filename, d10_seq_number):
    
    with open(d10_filename, 'r') as count_d10:
        d10_lines = count_d10.readlines()
        
    total_d10_lines = len(d10_lines)
    
    with open(d10_filename, 'r') as d10_body:
        d10_existing_content = d10_body.read()
        
    d10_header = f"""ZHV|{d10_seq_number}|D0010002|X|SUPPLIER|D|DC|{formatted_dt}||||OPER|\n"""
    d10_footer = f"""ZPT|{d10_seq_number}|{total_d10_lines}||{d10_counter}|{formatted_dt}|"""
        
    print(total_d10_lines)
        
    with open(d10_filename, 'w') as d10_headfoot:
        d10_headfoot.write(d10_header)
        d10_headfoot.write(d10_existing_content)
        d10_headfoot.write(d10_footer)
        

def delete_pre_script (directory):
    if os.path.exists(directory) and os.path.isdir(directory):
        outputs = os.listdir(directory)

        for file in outputs:
            file_path = os.path.join(directory, file)

            if os.path.isfile(file_path):
                os.remove(file_path)


def delete_empty_files(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and os.path.getsize(filepath) == 0:
            os.remove(filepath)

In [ ]:
# ***** file directories *****
print("^ File Directories + Containers + Counters")

d10s = []
flows.append(d10s)

# universal
d10_csv = '/home/jovyan/shared/Settlement/SSC Changer/D10_variables.csv'

# mrcy d10s
mrcy_d10_udms_output = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/D10s/MRCY_UDMS_GR{file_id_dt}0a.D0010'
mrcy_d10_accu_output = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/D10s/MRCY_ACCU_GR{file_id_dt}1a.D0010'

# rege d10s
rege_d10_udms_output = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/D10s/REGE_UDMS_GR{file_id_dt}2a.D0010'
rege_d10_accu_output = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/D10s/REGE_ACCU_GR{file_id_dt}3a.D0010'

# delete old .txts
delete_d10s = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/'

files = [mrcy_d10_udms_output, mrcy_d10_accu_output, rege_d10_udms_output, rege_d10_accu_output]

for file in files:
    open(file, 'x')
    
# ***** variables for flow header + footer *****
# counters for instances section of footers

# mrcy d10s
mrcy_d10_udms_counter = [0]
mrcy_d10_accu_counter = [0]
# rege d10s
rege_d10_udms_counter = [0]
rege_d10_accu_counter = [0]

In [ ]:
# dictionaries
print("^ Dictionaries")

# ***** d10 output dictionary *****
d10_dictionary = {
    'MRCY_UDMS': {'d10_body_output': mrcy_d10_udms_output, 'd10_body_counter': mrcy_d10_udms_counter},
    'MRCY_ACCU': {'d10_body_output': mrcy_d10_accu_output, 'd10_body_counter': mrcy_d10_accu_counter},
    'REGE_UDMS': {'d10_body_output': rege_d10_udms_output, 'd10_body_counter': rege_d10_udms_counter},
    'REGE_ACCU': {'d10_body_output': rege_d10_accu_output, 'd10_body_counter': rege_d10_accu_counter},
}

In [ ]:
print("Parse + Write D10s")

with open(d10_csv, 'r') as d10vars:
    reader = csv.reader(d10vars, delimiter=',')
    header = next(reader)
    rows = list(reader)
    
    for current_row, next_row in zip(rows, rows[1:]):
        MPAN = current_row[1]
        MSN = current_row[2]
        SUPPLIER = current_row[3]
        DC = current_row[4]
        EFD = current_row[5]
        EFD_datetime = dt.strptime(EFD, '%Y-%m-%d')
        formatted_EFD = EFD_datetime.strftime('%Y%m%d')
        REG = current_row[8]
        READ_1 = current_row[9]
        SSC = current_row[11]
        SSC = SSC.zfill(4)
        
        #with open(mpan_tracker, 'r') as read_tracker:
            #tracker_contents = read_tracker.read()
            
            #if MPAN in tracker_contents:
                #continue
        
        #with open(mpan_tracker, 'a') as tracker:
            #tracker.write(MPAN)
            #tracker.write('\n')
        
        if SSC == '0151':
            READ_2 = next_row[9]
        
        d10_key = f'{SUPPLIER}_{DC}'
      
        if d10_key in DC_dictionary:
            d10_dict_output = DC_dictionary[d10_key]
            d10_body_output = d10_dict_output['d10_body_output']
            d10_counter = d10_dict_output['d10_counter']

        if SSC == '0393':
            generate_single_d10_content(MPAN, MSN, formatted_EFD, d10_body_output, d10_counter, READ_1)
            
        if SSC == '0151':
            generate_dual_d10_content(MPAN, MSN, formatted_EFD, d10_body_output, d10_counter, READ_1, READ_2)

In [ ]:
directory_path_d10s = '/home/jovyan/shared/Settlement/SSC Changer/Outputs/D10s'
